# PICSUM- Deployment

## Installing the prerequisite libraries

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install transformers
!pip install gradio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Importing libraries

In [ ]:
import gradio as gr                               #For using gradio interface
from PIL import Image                             #For reading the image
import pytesseract                                #Used to read text from image
import torch
import numpy as np
import nltk                                       #Used to download stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance     # Used to find cosine distance between 2 vectors
import networkx as nx                             # Used for creating sentence similarity graph
from transformers import pipeline


if torch.cuda.is_available():
   device = torch.device("cuda")
else:
   device = torch.device("cpu")


summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Creating Functions

In [ ]:
def read(filepath):
    """
    Construct a image reading function via pytesseract

    Args:
        filepath (str)  : The filepath of the image which is to be read
    Return:
        text (str)      : The text extracted from the image

    """
    return pytesseract.image_to_string(Image.open(filepath))

def clean_text(text):
    """
    Returns clean text after preprocessing the image i.e. removing punctuations and extra spaces.

    Args:
        text (str)      : The text input which has to be cleaned
    Return:
        sentences (list): A list whose each element is a sentence
                          (A sentence is a list of words)

    """

    article = text.split(".")
    article=[sentence for sentence in article if sentence!=""]                #Removing unnecessary spaces

    sentences = []

    for sentence in article:
        sentence=sentence.replace(",", " , ").replace("'", " ' ").split(" ")  #Removing punctuations
        sentence=[word for word in sentence if word!=""]                      #Removing empty words
        sentences.append(sentence)

    return sentences

def sentence_similarity(sent1, sent2, stopwords):

    """
    Converts words in sentences to one hot encoding format and finds the cosine distance between
    the vectors inorder to measure closeness

    Args:
        sent1 (list)      : A list of string where each string represents a word in sentence
        sent2 (list)      : A list of string where each string represents a word in sentence
        stopwords (list)  : A list of commonly used words in English Voculbary
    Return:
        sentences (list): A list whose each element is a sentence
                          (A sentence is a list of words)

    """


    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]                        #Converting all words to lower case for uniformity
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))                      #Combining lists and removing duplicates

    vector1 = [0] * len(all_words)                            #Converting sentences to vectors
    vector2 = [0] * len(all_words)

    for w in sent1:                                           #Build the vector for the first sentence
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    for w in sent2:                                           #Build the vector for the second sentence
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    if np.isnan(1 - cosine_distance(vector1, vector2)):       #If the cosine vector produced nan values
        return 0

    return 1 - cosine_distance(vector1, vector2)              #Using cosine distance to measure similarity between 2 vectors


def build_similarity_matrix(sentences, stop_words):
    """
    Creates a matrix based on the similarity between 2 sentences

    Args:
        sentences (list)              : A list of sentence where each sentence represents a string of words
        stopwords (list)              : A list of commonly used words in English Voculbary
    Return:
        similarity_matrix (np.array)  : A 2-D array representing the similarity scores between 2 sentences

    """

    similarity_matrix = np.zeros((len(sentences), len(sentences)))    #Create an empty similarity matrix

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:                                          #Ignore if both are same sentences
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

    # Values closer to 0 have a higher correlation while values closer to 1 have a low correlation

def generate_important_sentences(text, top_n="auto"):
    """
    Picking out top_n sentences from the given text

    Args:
        text  (str)             : The text extracted from the image
        top_n (str/int)         : Number of sentences to generate. Default is "auto" which generates all
                                  possible sentences in order of their priority
    Return:
        extractive_summarized (str)  : A string denoting the top_n important sentences.

    """
    # Step 1 - Clean text to generate sentences

    sentences=clean_text(text)
    stop_words = stopwords.words('english')
    stop_words.append(".")
    stop_words.append(",")
    summarize_text = []

    # Step 2 - Generate Similary Martix across sentences

    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix

    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)

    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank (by scores in descending order) and pick top sentences

    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)


    if top_n=="auto": top_n=len(ranked_sentence)
    else: top_n=int(top_n)

    for i in range(top_n):
      ranked_sentence[i][1][0]=ranked_sentence[i][1][0].capitalize()    #Capitalising 1st letter of sentence
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarized text

    extractive_summarized=". ".join(summarize_text).replace(" , ",", ").replace(" ' ","'") + "."
    return extractive_summarized

def important_sentences(filepath, no_of_sentences=5):
    """
    Picking out no_of_sentences from the given text and displaying them inoder

    Args:
        filepath (str)              : The filepath of the image which is to be read
        no_of_sentences (int)       : Number of sentences to generate. Default is 5
    Return:
        tuple                       : Gradio commands to print output

    """

    extractedInformation=read(filepath)                                   #reading the information from filepath
    extractedInformation=' '.join(extractedInformation.split('\n'))

    try:                                                                  #if the text can generate 5 sentences
      extractive_summary=generate_important_sentences(extractedInformation, no_of_sentences)
    except:                                                               #if not enough information, generate max possible number of sentences
      extractive_summary=generate_important_sentences(extractedInformation,"auto")

    text=""
    for index,sent in enumerate(extractive_summary.split(".")):
      if sent!='':text+=str(index+1)+". "+str(sent).strip()+".\n\n"
    return (gr.Textbox.update(text),gr.Button.update(visible=False),gr.Textbox.update(visible=False),gr.Dropdown.update(visible=False))

def summarize(filepath):
    """
    Summarize the information from image

    Args:
        filepath (str)              : The filepath of the image which is to be read
    Return:
        tuple                       : Gradio commands to print output

    """

    extractedInformation=read(filepath)                                  #extracting information from filepath
    extractedInformation=' '.join(extractedInformation.split('\n'))
                                                                          #generating an abstractive summary with dynamic length
    abstractive_summary = summarizer(extractedInformation, max_length=int(len(extractedInformation)/6), min_length=int(len(extractedInformation)/10), do_sample=False)
    return (gr.Textbox.update(abstractive_summary[0]["summary_text"]),gr.Button.update(visible=False),gr.Textbox.update(visible=False),gr.Dropdown.update(visible=False))

def Question_Answer(filepath,question,mode):
    """
    Generate question answer from the given context

    Args:
        filepath (str)              : The filepath of the image which is to be read
        question (str)              : The question to be answered
        mode (str)                  : Whether Roberta model to use or distilbert
    Return:
        tuple                       : Gradio commands to print output

    """
    extractedInformation=read(filepath)
    extractedInformation=' '.join(extractedInformation.split('\n'))
    if mode=="Roberta":
      question_answerer = pipeline("question-answering", model="SMD00/QA_model-roberta")
    else :
      question_answerer = pipeline("question-answering", model="SMD00/QA_model-distilbert")
    obj=question_answerer(question=question, context=extractedInformation)
    return obj['answer']


## Creating Gradio Interface

In [ ]:
def show_fn():
  return (gr.Textbox.update(visible=True),gr.Button.update(visible=True),gr.Dropdown.update(visible=True),gr.Textbox.update(""))
def dummy_fn(x):
  return x

with gr.Blocks() as demo:
    gr.Markdown("# **PicSum**")
    gr.Markdown("Gradio demo for PicSum project. You can give an image as input and select any of the three buttons. It generates summary, important sentences and answers questions related to context.")
    img=gr.components.Image(type="filepath", label="Input Image")

    with gr.Row():
        summary_btn = gr.Button(value="Summary")
        sentence_btn = gr.Button(value="Important Sentences")
        quesAndAns_btn = gr.Button(value="Question and Answers")

    mode=gr.Dropdown(["Roberta","DistilBert"],label="Model",info="Choose a model",visible=False)
    ques_box = gr.Textbox(label="Question",info="Enter a Question",interactive=True,visible=False)
    submit_btn= gr.Button(value="Submit",visible=False)
    out_box=gr.Textbox(label="Generated Text")
    summary_btn.click(fn=summarize,inputs=[img],outputs=[out_box,submit_btn,ques_box,mode])
    sentence_btn.click(fn=important_sentences,inputs=[img],outputs=[out_box,submit_btn,ques_box,mode])
    quesAndAns_btn.click(fn=show_fn,outputs=[submit_btn,ques_box,mode,out_box])
    submit_btn.click(fn=Question_Answer,inputs=[img,ques_box,mode],outputs=[out_box])
    gr.Markdown("## Image Examples")
    with gr.Row():
        gr.Examples(
            examples=[ "/content/a.png"],
            inputs=img,
            outputs=img,
            fn=dummy_fn,
            cache_examples=True,
        )
        gr.Examples(
            examples=[ "/content/b.png"],
            inputs=img,
            outputs=img,
            fn=dummy_fn,
            cache_examples=True,
        )
demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:261: UserWarning: api_name load_example already exists, using load_example_1
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")


Using cache from '/content/gradio_cached_examples/14' directory. If method or examples have changed since last caching, delete this folder to clear cache.
Using cache from '/content/gradio_cached_examples/15' directory. If method or examples have changed since last caching, delete this folder to clear cache.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
